In [ ]:
# ! pip install autokeras

In [ ]:
import pandas as pd
import numpy as np
import itertools
# import matplotlib.pyplot as plt
import useful_functions as uf
# from sklearn.preprocessing import OneHotEncoder
path_fill = '/workspaces/CMB_match/data/2022/filled_trian.csv'
path_train_pure = '/workspaces/CMB_match/data/2022/train.csv'
# data/2022/filled_trian.csv
df_train_filled = pd.read_csv(path_fill, index_col=0)
df_train = pd.read_csv(path_train_pure)

有序型分类变量：使用LabelEncoder()编码；

无序型分类变量：使用独热编码。

In [ ]:
string_col =uf.dynamic_string_col(df_train_filled)

opt_col = string_col.copy()

df_str=pd.DataFrame()
if 'MON_12_CUST_CNT_PTY_ID' in string_col:
    # This a Binary Y/N, the training sample has Y only, do not fill with the mode
    opt_col.remove('MON_12_CUST_CNT_PTY_ID')
    df_str.loc[:,'MON_12_CUST_CNT_PTY_ID'] = df_train_filled['MON_12_CUST_CNT_PTY_ID']
    # Direct copy
# fill the missing value with mode
df_str.loc[:,opt_col] = uf.mode_nan_string(df_train_filled.loc[:, opt_col])
## One-hot encoding
df_str.loc[:,'SHH_BCK']=df_str.SHH_BCK.astype(str) # it was floated
dummies = pd.get_dummies(df_str,dummy_na=True)


In [ ]:
_df_numerical= uf.get_numerical_df(df_train_filled)

1 无序分类变量的相关性
2 get the correlation matrix of the numerical variables
3 save the first column with the lowest missing value

In [ ]:
corr_cols = uf.high_corr_col(df_train_filled)

In [ ]:

drop_col = uf.save_n_drop(corr_cols,df_train_filled)
test = df_train_filled.drop(drop_col,axis=1)
second_corr = test.loc[:,uf.high_corr_col(test)].corr()
second_corr

There are still four columns i.e. two paris are high correlated variables

In [ ]:
rows,cols = np.where(second_corr>0.8)
# gives the position of all correlation value >0.8
get_rid_2=list()
for i in range(len(rows)):
    posi= second_corr.iloc[rows[i],cols[i]]
    # rows[i],cols[i] is the position of the element in the matrix
    # posi is the correlation value of rows[i] and cols[i]
    if posi<0.9990: # type: ignore
        col_set = {second_corr.index[rows[i]], second_corr.columns[cols[i]]}
        # this is the actual column name
        if col_set not in get_rid_2: # because the set disregard the order,
            # so we need to check if the set is already in the list,
            # the pair with the same two elements
            # but different order will be regarded as same set,
            get_rid_2.append(col_set)
drop_col_2  = list(itertools.chain(*get_rid_2)) # take component out to list
# drop_col_2 is the list of column name that need to be dropped

两遍去，第一遍按前三字一组取最低缺失，第二次按照pair取

In [ ]:
rank_df = pd.DataFrame(df_train.loc[:, drop_col_2].isna().sum())
#---> index:column name, value: number of missing value
rank_df.sort_values([0], ascending=True)

4 use the drop_col we get to drop again

In [ ]:

for i in range(len(get_rid_2)):# loop in length of the four element list
    for col_j in get_rid_2[i]: # col_j is the column name
        rank_df.loc[col_j,'pair']=i # add a column of pair mark to rank_df
        rank_df.loc[col_j,'col_name']=col_j # add a column of column name to rank_df
ndarray=rank_df.groupby('pair')[0].nlargest(1)
ndarray
# rank_df.loc[] # find the column with the least missing value
# df.nlargest()
# [col for col in test.columns if col not in ndarray]

In [92]:
rows,cols = np.where(second_corr>0.8)
# gives the position of all correlation value >0.8
get_rid_2=list()
for i in range(len(rows)):
    posi= second_corr.iloc[rows[i],cols[i]]
    # rows[i],cols[i] is the position of the element in the matrix
    # posi is the correlation value of rows[i] and cols[i]
    if posi<0.9990: # type: ignore
        col_set = {second_corr.index[rows[i]], second_corr.columns[cols[i]]}
        # this is the actual column name
        if col_set not in get_rid_2: # because the set disregard the order,
            # so we need to check if the set is already in the list,
            # the pair with the same two elements
            # but different order will be regarded as same set,
            get_rid_2.append(col_set)
drop_col_2  = list(itertools.chain(*get_rid_2)) # take component out to list
# drop_col_2 is the list of column name that need to be dropped

两遍去，第一遍按前三字一组取最低缺失，第二次按照pair取

In [93]:
rank_df = pd.DataFrame(df_train.loc[:, drop_col_2].isna().sum())
#---> index:column name, value: number of missing value
rank_df.sort_values([0], ascending=True)

,0
EMP_NBR,106
HLD_DMS_CCY_ACT_NBR,114
LAST_12_MON_COR_DPS_DAY_AVG_BAL,114
CUR_MON_COR_DPS_MON_DAY_AVG_BAL,116


4 use the drop_col we get to drop again

In [94]:

for i in range(len(get_rid_2)):# loop in length of the four element list
    for col_j in get_rid_2[i]: # col_j is the column name
        rank_df.loc[col_j,'pair']=i # add a column of pair mark to rank_df
        rank_df.loc[col_j,'col_name']=col_j # add a column of column name to rank_df
ndarray=rank_df.groupby('pair')[0].nlargest(1)
ndarray
# rank_df.loc[] # find the column with the least missing value
# df.nlargest()
# [col for col in test.columns if col not in ndarray]

pair                                 
0.0   HLD_DMS_CCY_ACT_NBR                114
1.0   CUR_MON_COR_DPS_MON_DAY_AVG_BAL    116
Name: 0, dtype: int64